In [ ]:
import cv2, os, joblib
import numpy as np
from pathlib import Path
from keras.models import load_model

In [ ]:
def Learning():
    import cv2
    from pathlib import Path
    from keras.models import Model, Sequential
    from keras.preprocessing import image
    from keras.applications import vgg16
    from keras.preprocessing.image import ImageDataGenerator
    from keras.layers import Dense, Conv2D, GlobalAveragePooling2D, Dropout, Activation, Flatten
    from keras.optimizers import Adam


    img_rows, img_cols = 224, 224 
    base_model =vgg16.VGG16(weights = 'imagenet', 
                     include_top = False, 
                     input_shape = (img_rows, img_cols, 3))


    # Here we freeze the last 4 layers 
    # Layers are set to trainable as True by default
    for layer in base_model.layers:
        layer.trainable = False
    #model.summary()


    def add(BottomModel, Class):
        x = BottomModel.output
        x = GlobalAveragePooling2D()(x)
        x = Flatten()(x)
        x = Dense(1024,activation='relu')(x)
        x = Dense(1024,activation='relu')(x)
        x = Dense(512,activation='relu')(x)
        x = Dense(Class,activation='softmax')(x)
        return x


    Class = 2
    top = add(base_model, Class)
    model = Model(inputs = base_model.input, outputs = top)


    #model.summary()


    dir = Path('C://Users//kevin//Documents//Project//Task_4//cell_images')
    #Image Data Generator
    train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
            dir / 'training_set',
            target_size=(img_rows, img_cols),
            batch_size=16,
            class_mode='categorical')

    
    validation_generator = validation_datagen.flow_from_directory(
        dir / 'testing_set',
        target_size=(img_rows, img_cols),
        batch_size=16,
        class_mode='categorical')

    # We use a very small learning rate 
    optimizer = Adam(lr=0.001)
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizer,
                  metrics = ['accuracy'])

    #validation_steps < validation_dataset_size / batch_size
    history = model.fit_generator(
    train_generator,
    epochs = 5,
    validation_data = validation_generator,
    )

    model.save('TL_Model.h5')

In [ ]:
def Test():    
    import os, random
    from PIL import Image, ImageFont, ImageDraw
    from pathlib import Path
    from keras.models import load_model
    from keras.preprocessing import image

    path = Path("C://Users//kevin//Documents//Project//Task_4//cell_images//prediction_set")
    fold = random.choice(os.listdir(path)) 
    im = Image.open(path / fold)
    im = im.resize((224,224), resample = Image.NEAREST)
    npimage = image.img_to_array(im)
    npimage = np.expand_dims(npimage, axis=0)

    model = load_model('TL_Model.h5')
    
    result = model.predict(npimage)
    if result[0][0] == 1.0:
        text = "Parasitized Cell"
        pil(text,im)
    else:
        text = "Healthy Cell"
        pil(text,im)

def pil(text, im):
    import string, random
    from pathlib import Path
    from PIL import Image, ImageFont, ImageDraw
    
    rand = random.choice(string.ascii_letters)
    
    d = ImageDraw.Draw(im)
    d.text((10,10), text, fill=(255,255,0))
    sv = 'pil_text_image' + rand + '.png'
    im.save(sv)
    newim = Image.open(sv)
    newim.show()

In [ ]:
def imgclassify(path):
    from keras.applications import Xception
    from keras.applications.inception_v3 import preprocess_input, decode_predictions
    from keras.utils.vis_utils import plot_model
    from keras.preprocessing.image import load_img, img_to_array
    from pathlib import Path

    model = Xception()
    
    directory = Path(path)
    # load an image from file
    image = load_img(directory, target_size=(299, 299))

    # convert the image pixels to a numpy array
    image = img_to_array(image)

    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

    # prepare the image for the model
    image = preprocess_input(image)

    # predict the probability across all output classes
    yhat = model.predict(image)

    # convert the probabilities to class labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # print the classification
    print('\033[91m' + '%s (%.2f%%)' % (label[1], label[2]*100) + '\033[0m')

In [ ]:
Choice = True
while Choice:
    print('\033[1m' + """
    1. Train Transfer Learning Model
    2. Test Transfer Learning Model
    3. Image Classification
    4.. Exit
    """)
    Choice = input("What would you like to do? ")

    if Choice == "1":
        Learning()
        print("\n \033[1m" + 'Model Trained')
    elif Choice == "2":
        Test()
        print("\n \033[1m" + "Test Successful")
    elif Choice == "3":
        path = input("Enter Full Path of Image Name With Extention")
        imgclassify(path)
        print("\n \033[1m" + "Image Successfully Classified")
    elif Choice == "4":
        print("\n \033[1m" + "Goodbye") 
        Choice = None
    else:
        print("\n \033[1m" + "Not Valid Choice Try again")